In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from keras.models import model_from_json
from PIL import Image
from keras import backend as K

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
# def check_dicom(filename): 
#     # todo
#     print('Load file {} ...'.format(filename))
#     ds = pydicom.dcmread(filename)      
#     print(ds.StudyDescription)
#     img = ds.pixel_array
#     return img
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = (img - img_mean) / img_std
    #print(f'type(img)={type(proc_img)}, img.shape={proc_img.shape}')
    proc_img = resize_image(proc_img, img_size[1:3])
    #print(f'type(img)={type(proc_img)}, img.shape={proc_img.shape}')
    return K.constant(proc_img)

def resize_image(img, img_size):
    rgbimg = Image.new("RGB", img_size)
    rgbimg.paste(Image.fromarray(img))
    return np.array(rgbimg.resize(img_size))[None,:]

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path, img_size):
    # todo
    with open(model_path, 'r') as f:
        model = model_from_json(f.read())
        model.build(img_size)
        #model.summary()
        model.load_weights(weight_path)    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo
    prediction = model(img) >= thresh
    return K.eval(prediction)[0][0]

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'model.json' #path to saved model
weight_path = 'best_weights.hdf5' #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0. # loads the mean image value they used during training preprocessing
img_std = 1. # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path, IMG_SIZE) #loads model
print('Model was loaded successfully!')
thresh = 0.4412 #loads the threshold they chose for model classification 

def has_pneumonia(study_description):
    return study_description.find('Pneumonia') >= 0

# use the .dcm files to test your prediction
correct_predictions = 0.
for dicom_file in test_dicoms:
    ds = pydicom.dcmread(dicom_file)
    study_description = ds.StudyDescription
    ground_truth = has_pneumonia(study_description)
    img = ds.pixel_array
    if img is None: continue
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    prediction = predict_image(my_model,img_proc,thresh)
    correct = ground_truth == prediction
    correct_predictions += correct
    correct_str = 'CORRECT' if correct else 'WRONG'
    print(f'Study description: {study_description}, ground_truth={ground_truth}, prediction={prediction} ({correct_str})')
acc = correct_predictions / len(test_dicoms)
print(f'ACCURACY={acc * 100:.2f}%')

Model was loaded successfully!
Study description: No Finding, ground_truth=False, prediction=False (CORRECT)
Study description: Cardiomegaly, ground_truth=False, prediction=False (CORRECT)
Study description: Effusion, ground_truth=False, prediction=False (CORRECT)
Study description: No Finding, ground_truth=False, prediction=False (CORRECT)
Study description: No Finding, ground_truth=False, prediction=False (CORRECT)
Study description: No Finding, ground_truth=False, prediction=False (CORRECT)
ACCURACY=100.00%


In [4]:
for dicom_file in test_dicoms:
    ds = pydicom.dcmread(dicom_file)
    print(f'\n{dicom_file}')
    print(ds)


test1.dcm
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) 